In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import pickle
from collections import defaultdict
import numpy as np
import scipy
import os
import time       
import os 

from emb.infersent import InferSent
from utils.find_ranking_citation import CitationRanking
from utils.write_result import WriteResult
from utils.utils import sort_this
from utils.configuraiton import Rec_configuration 
from utils.eval import Metrics
from utils.data_loading import DataLoading

In [ ]:
class Master:
    def __init__(self, rerank = False, V=2 ):
        self.rec_conf = Rec_configuration()
        self.article_title, self.article_abstract, self.geo_title, \
        self.geo_summary, self.citation_data = DataLoading().get_all_details() #we should get actual citations
        self.auto_rank = CitationRanking(self.citation_data)
        self.write_res = WriteResult()
        choose = list(self.citation_data.keys())
        self.geo_test_data = [key for key in choose if self.citation_data[key] != []] #76,064
        #self.rr = ReRankingTitle() # i definitely need to work on this as well (re-ranking title using bert)
        self.top_threshold = 10 #sounds nice
        self.rerank = rerank
        if V==1:
            self.model_addr = self.rec_conf.model_path_inferSent1 #subject to change
            self.voc = self.rec_conf.W2V_PATH1
        else:
            self.model_addr = self.rec_conf.model_path_inferSent2 #subject to change
            self.voc = self.rec_conf.W2V_PATH2
        self.params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
        self.res_addr = self.rec_conf.result_address_inferSent #subject to change
        self.use_cuda =  torch.cuda.is_available()
        

    def process(self):
        #check paths and check vecs  
        infersent = InferSent(self.params_model)
        infersent.load_state_dict(torch.load(self.model_addr))
        infersent.set_w2v_path(self.voc)
        infersent = infersent.cuda() if self.use_cuda else infersent
        pmids, infersent_vecs = None, None #this should be loading something

        # This is for loading the pre-trained bert models.
        if infersent_vecs is not None and pmids is not None:
            print('load trained models and encoded publications vectors')
            '''
            
            '''
        else:
            print('Building models')
            joined_dict_article = {}
            for pmid in self.article_title: #for articles, encoding the articles
                #print('pmid')
                #print(pmid)
                if (self.article_title[pmid] + self.article_abstract[pmid]).strip() != '':
                    joined_dict_article[pmid] = self.article_title[pmid] + ' ' + self.article_abstract[pmid]
            #should be some training here, otherwise just do the easiest encodinh
            infersent.build_vocab(list(joined_dict_article.values()), tokenize=True) #vocabulary 8660 
            pmids, infersent_vecs = list(joined_dict_article.keys()), infersent.encode(list(joined_dict_article.values()), bsize= 64, 
                            verbose=True)#vocabulary 8660 

            with open(self.res_addr + 'smallbase/infersent_vecs', 'wb') as handle:
                pickle.dump(infersent_vecs, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open(self.res_addr + 'smallbase/pmids', 'wb') as handle:
                pickle.dump(pmids, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
        joined_dict_geo = {} # maxium: 76,064
        # screening first for those geo_ids with pmids 
        for geo_id in self.geo_test_data:
            print('geo_id')
            print(geo_id)
            if (self.geo_title[geo_id] + self.geo_summary[geo_id]).strip() != '':
                #only get the ones with actual texts
                joined_dict_geo[geo_id] = self.geo_title[geo_id] + ' ' + self.geo_summary[geo_id]      
        geo_ids, geo_infersent_vecs = list(joined_dict_geo.keys()), infersent.encode(list(joined_dict_geo.values()), bsize= 64, verbose=True)
        print('nb sentences encoded : {0}'.format(len(geo_infersent_vecs)))       
        #geo_w2v_vecs_dict = w2v.create_vectors(joined_dict_geo)
        with open(self.res_addr + 'smallbase/geo_infersent_vecs ', 'wb') as handle:
            pickle.dump(geo_infersent_vecs, handle, protocol=pickle.HIGHEST_PROTOCOL)   
        with open(self.res_addr + 'smallbase/geo_ids', 'wb') as handle:
            pickle.dump(geo_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)   


In [ ]:
def main():
    x = Master(rerank = False, V=2)
    x.process()


if __name__ == '__main__':
    main()

In [ ]:
#read-in stored vec 
res_addr = 'results/inferSent_plain/smallbase/'
model_addr = 'resources/inferSent/'
with open(res_addr +'infersent_vecs', 'rb') as fp:
    infersent_vecs = pickle.load(fp)
    
with open(res_addr +'geo_infersent_vecs ', 'rb') as fp:
      geo_infersent_vecs = pickle.load(fp)        
        
with open(res_addr +'pmids', 'rb') as fp:
      pmids = pickle.load(fp)
with open(res_addr +'geo_ids', 'rb') as fp:
      geo_ids = pickle.load(fp)      

In [ ]:
#check 
print(geo_infersent_vecs.shape) #72791 * 4096
print(infersent_vecs.shape) #50159
#infersent_vecs to numpy first?
print(type(geo_infersent_vecs))
print(type(infersent_vecs))

In [ ]:
_, _, _, _, citation_data = DataLoading().get_all_details() #we should get actual citations
auto_rank = CitationRanking(citation_data)
write_res = WriteResult()  
b_size = 64
#tfidf_vecs = tfidf_vecs.toarray()

In [ ]:

chunks = len(geo_infersent_vecs) // b_size +1
for step in range(chunks):
    batch = geo_infersent_vecs[step*b_size:(step+1)*b_size]
    #batch = np.array(batch) #?
    print(batch.shape)
    geo_ids_batch = geo_ids[step*b_size: (step+1)*b_size]
    #geo_batch, geo_ids_batch = batch
    similarity_value_dict = dict()
    similarity_scores = cosine_similarity(batch, infersent_vecs)
    np.save(res_addr + 'similarity_scores_batch_' + str(step), similarity_scores)#.cpu().numpy())
    #load this later
    #np.load(self.res_addr + 'base/similarity_scores')
    print(similarity_scores.shape)
    #call the re-ranking or not 
    for i, geo_id in enumerate(geo_ids_batch): #this is only from testing geo_id
        print(geo_id)
        similarity_value_dict[geo_id] =  list(similarity_scores[i])#.cpu().numpy()

    new_similarity_dict =  sort_this(geo_sim_dict = similarity_value_dict, pmid_ls = pmids)
    new_similarity_dict = dict(new_similarity_dict)
    filename =  res_addr + 'new_similarity_dict_batch_' + str(step)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(res_addr + 'new_similarity_dict_batch_' + str(step), 'wb') as handle:
        pickle.dump(new_similarity_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    print('export dict!!!!!!!!')
     
    for id, geo_id in enumerate(geo_ids_batch): 
        print(geo_id)
        #this has already stored in a pickled list of geo_haspmid.pickle
        temp_selected = new_similarity_dict[geo_id]
        filename =  res_addr + geo_id + '.txt'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        write_res.write(res_addr  + geo_id + '.txt', temp_selected)
        auto_rank.find_citations(geo_id, list(temp_selected.keys())) #keys are a list of pmids

a, b, c, d = auto_rank.get_values() #need to modify this 
print('good geo recommendations = {}, top1 hit geo recommendations = {}, bad geo recommendations = {}, '
      'geo without citations = {}'.format(a, b, c, d))

In [ ]:
#import glob
#files = glob.glob(res_addr + "similarity_scores_batch_*")
res_addr = 'results/inferSent_plain/smallbase/'
new_similarity_dict =  {}
#print
#for file in files:
for i in range(step +1):
    with open(res_addr + "new_similarity_dict_batch_" + str(i), 'rb') as fp:
        dict_batch = pickle.load(fp)
        new_similarity_dict.update(dict_batch)
#dict.update(dict2)
#dict.update(dict3)
with open(res_addr + 'new_similarity_dict.pickle', 'wb') as handle:
    pickle.dump(new_similarity_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#MRR 
print('MRR:')
print(Metrics(citation_data).calculate_mrr(new_similarity_dict)) #mrr
#recall @1 and recall @10
print('recall@1, recall@10:')
print(Metrics(citation_data).calculate_recall_at_k(new_similarity_dict, 1))
print(Metrics(citation_data).calculate_recall_at_k(new_similarity_dict, 10))
#Precision@1 and precision10
print('precision@1, precision@10:')
print(Metrics(citation_data).calculate_precision_at_k(new_similarity_dict, 1))        
print(Metrics(citation_data).calculate_precision_at_k(new_similarity_dict, 10))
#MAP@10
print(Metrics(citation_data).calculate_MAP_at_k(new_similarity_dict))